In [1]:
%load_ext autoreload
%autoreload 2

from stable_audio_tools.models.autoencoders import AudioConditionalVariationalAutoEncoder
from stable_audio_tools.models import create_model_from_config
from stable_audio_tools.models.utils import copy_state_dict
from stable_audio_tools.data.utils import PadCrop_Normalized_T, Stereo, Mono

import torch
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms as T
from pathlib import Path
from IPython.display import Audio
from tqdm.auto import tqdm

from pathlib import Path

loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-conditional-vae/fky8t0jw/checkpoints/epoch=80-step=50000.ckpt")

/tmp/ipykernel_1096845/1199835435.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-conditio

In [2]:
pad_crop = PadCrop_Normalized_T(n_samples=65536, sample_rate=44100)
to_stereo = Stereo()

In [3]:
state_dict = {}
for key in loaded['state_dict']:
    if "autoencoder" in key:
        keyy = key.replace("autoencoder.", "")
        state_dict[keyy] = loaded['state_dict'][key]

model = create_model_from_config(loaded['model_config'])
copy_state_dict(model, state_dict)
model.eval()
model.to("cuda:0")

/home/sake/.local/share/virtualenvs/stable-audio-cvae-diFMr3O8/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


AudioConditionalVariationalAutoEncoder(
  (bottleneck): VAEBottleneck()
  (encoder): DACEncoderWrapper(
    (encoder): Encoder(
      (block): Sequential(
        (0): Conv1d(2, 128, kernel_size=(7,), stride=(1,), padding=(3,))
        (1): EncoderBlock(
          (block): Sequential(
            (0): ResidualUnit(
              (block): Sequential(
                (0): Snake1d()
                (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
                (2): Snake1d()
                (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
              )
            )
            (1): ResidualUnit(
              (block): Sequential(
                (0): Snake1d()
                (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
                (2): Snake1d()
                (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
              )
            )
            (2): ResidualUnit(
              (block): Sequential(
             

In [4]:
wav_fns = list(Path('/home/sake/userdata/sake/oneshot_data').rglob('*.wav'))
len(wav_fns)

16328

In [5]:
labels = {
    'kick': 0,
    'snare': 1,
    'hat': 2,
    'clap': 3,
    'percussion': 4
}

latents = {
    'kick': {},
    'snare': {},
    'hat': {},
    'clap': {},
    'percussion': {}
}

for wav_fn in tqdm(wav_fns):
    classname = wav_fn.parent.stem
    class_int = labels[classname]
    class_emb = torch.tensor([class_int]).unsqueeze(0).to("cuda:0")
    
    wav, sr = torchaudio.load(wav_fn)
    if sr != 44100:
        resample_tf = T.Resample(sr, 44100)
        wav = resample_tf(wav)

    wav, t_start, t_end, seconds_start, seconds_total, padding_mask = pad_crop(wav)
    wav = wav.clamp(-1, 1)
    wav = to_stereo(wav)

    with torch.inference_mode():
        emb = model.encode(wav.unsqueeze(0).to("cuda:0"), condition=class_emb, skip_bottleneck=True)
        mean = emb.mean()
        std = emb.std()
        emb = model.bottleneck.encode(emb)

    latents[classname][wav_fn.name] = {
        'embedding': emb.squeeze().cpu(),
        'mean': mean,
        'std': std
    }


torch.save(latents, 'latents-epoch=80-step=50000.pt')

  0%|          | 0/16328 [00:00<?, ?it/s]